In [13]:
import pandas as pd

df = pd.read_csv("../../../data/WV/09-28-22/23-5-24/active-inactive-wv-le--20231220144255.csv")

In [14]:
df = df.rename(columns={"Person LEPS Subcommittee ID": "person_nbr"})

df

,person_nbr,Person First Name,Person Last Name,Person UDF - Date Certified as an Officer,Employment Start Date,Employment End Date,Employment Appointment Type,Employment Employment Type,Employment Title/Rank (Current),Employment Status,...,Employment Start Date.6,Employment End Date.6,Employment Appointment Type.6,Employment Employment Type.6,Employment Title/Rank (Current).6,Employment Status.6,Employing Organization Name.6,Employment Start Date.7,Employment End Date.7,Employment Appointment Type.7
0,0880-0382,Chad,Clatterbuck,4/27/2007,12/18/2005,NaN,Sworn/Certified,Full Time,Corporal,Active,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3412-3729,Michael,Billiter,4/27/2007,12/19/2005,7/3/2013,Sworn/Certified,NaN,Officer,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6031-5728,William,Kearns,NaN,12/13/2005,6/4/2007,NaN,NaN,Officer,Resigned - Status Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2947-7452,Basra,Fakhir,8/25/2006,9/9/2005,6/17/2014,Sworn/Certified,NaN,Officer,Retired - In Good Standing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0818-5384,Roland,Houck,NaN,7/1/2005,4/6/2016,NaN,NaN,Officer,Other,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13299,1473-7900,Chase,Lambert,NaN,12/15/2022,1/4/2023,Sworn Pending Certification,Full Time,Officer,Resigned - In Good Standing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13300,7475-8802,Samuel,Brake,NaN,11/1/2022,1/2/2023,Sworn Pending Certification,Full Time,Officer,Resigned - In Good Standing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13301,9025-7364,Zachary,Lynn,NaN,3/13/2023,NaN,Non-Sworn,Full Time,Civilian,Active,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13302,1893-3623,Nathaniel,Murphy,NaN,11/3/2022,NaN,Sworn/Certified,Full Time,Officer,Active,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Identify the columns that don't change across employments
id_cols = ['person_nbr', 'Person First Name', 'Person Last Name', 'Person UDF - Date Certified as an Officer']

# Identify the columns that change for each employment
var_cols = ['Employment Start Date', 'Employment End Date', 'Employment Appointment Type', 
            'Employment Employment Type', 'Employment Title/Rank (Current)', 'Employment Status', 
            'Employing Organization Name']

# Create a list of all variable columns, including those with suffixes
all_var_cols = [col for col in df.columns if any(var_col in col for var_col in var_cols)]

# Melt the DataFrame
melted = pd.melt(df, id_vars=id_cols, value_vars=[col for col in all_var_cols], 
                 var_name='variable', value_name='value')

# Extract the employment number and original column name
melted['emp_num'] = melted['variable'].str.extract('(\d+)').fillna('0').astype(int)
melted['original_column'] = melted['variable'].str.replace(r'\.\d+$', '', regex=True)

# Pivot the melted DataFrame
reshaped = melted.pivot_table(index=id_cols + ['emp_num'], columns='original_column', values='value', aggfunc='first').reset_index()

# Clean up column names
reshaped.columns.name = None

# Sort the DataFrame
reshaped = reshaped.sort_values(['person_nbr', 'emp_num'])

# Reset index for clean output
reshaped = reshaped.reset_index(drop=True)

reshaped = reshaped.rename(columns={"Employment Start Date": "start_date", 
                                    "Employment End Date": "end_date", 
                                    "Employment Status": "separation_reason",
                                    "Employment Title/Rank (Current)": "rank",
                                    "Person UDF - Date Certified as an Officer": "initial_certification_date",
                                    "Person First Name": "first_name",
                                    "Person Last Name": "last_name",
                                    "Employing Organization Name": "agcy_name",
                                    "Employment Employment Type": "employment_type", 
                                    "Employment Appointment Type": "certified"})

reshaped = reshaped.drop(columns=["emp_num"])

def clean_separation_reason(df):
    df.loc[:, "separation_reason"] = (df.separation_reason
                                        .str.replace(r"Resigned - (.+)", r"Resigned \1", regex=True)
                                        .str.replace(r"Retired - (.+)", r"Retired \1", regex=True)
                                        .str.replace(r"Terminated - (.+)", r"Terminated \1", regex=True)
    )
    return df 




def drop_fire_dept_rows(df):
    df = df[~(df.agcy_name.fillna("").str.lower().str.contains(r"fire"))]
    return df 


reshaped = reshaped.pipe(clean_separation_reason).pipe(drop_fire_dept_rows)

reshaped.loc[:, "full_name"] = reshaped.first_name.str.cat(reshaped.last_name, sep=" ")


reshaped = reshaped.drop(columns=["employment_type"])

def upper_cols(df):
    for col in df.columns:
        df = df.apply(lambda col: col.astype(str).str.upper())

    return df 

reshaped = reshaped.pipe(upper_cols)

reshaped

<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/var/folders/4r/jhc99db14v58mjmncgtry0kc0000gn/T/ipykernel_3224/3130391517.py:17: SyntaxWarning: invalid escape sequence '\d'
  melted['emp_num'] = melted['variable'].str.extract('(\d+)').fillna('0').astype(int)


,person_nbr,first_name,last_name,initial_certification_date,agcy_name,certified,end_date,start_date,separation_reason,rank,full_name
0,0001-7902,JOHN,SYNER,4/9/2004,WEST VIRGINIA STATE POLICE,SWORN/CERTIFIED,5/1/2020,9/29/2003,RESIGNED IN GOOD STANDING,OFFICER,JOHN SYNER
1,0002-3178,GERALD,PAULEY,8/20/1999,PUTNAM COUNTY SHERIFF'S OFFICE,NAN,5/31/2006,9/15/1998,RESIGNED STATUS UNKNOWN,OFFICER,GERALD PAULEY
2,0002-3178,GERALD,PAULEY,8/20/1999,CERTIFICATION RETENTION POLICE DEPT,SWORN/CERTIFIED,3/17/2018,7/1/2014,RESIGNED IN GOOD STANDING,CIVILIAN,GERALD PAULEY
3,0002-3178,GERALD,PAULEY,8/20/1999,KANAWHA COUNTY PARKS POLICE,SWORN/CERTIFIED,NAN,3/17/2018,ACTIVE,PATROLMAN,GERALD PAULEY
4,0003-3513,ANTHONY,BISE,4/28/2006,MONONGALIA COUNTY SHERIFF'S OFFICE,SWORN/CERTIFIED,7/11/2021,10/15/2005,RETIRED IN GOOD STANDING,SERGEANT,ANTHONY BISE
...,...,...,...,...,...,...,...,...,...,...,...
11423,9993-3822,LESLIE,BAILEY,10/14/1986,RALEIGH COUNTY SHERIFF'S OFFICE,NAN,7/16/1986,7/15/1986,ASSUMED SEPARATED (END DATE UNKNOWN),OFFICER,LESLIE BAILEY
11424,9993-3822,LESLIE,BAILEY,10/14/1986,WEST VIRGINIA STATE POLICE,NAN,3/30/2012,1/1/1901,RETIRED STATUS UNKNOWN,OFFICER,LESLIE BAILEY
11425,9996-4096,MICHAEL,RANDALL,1/6/2006,PARKERSBURG POLICE DEPARTMENT,NAN,11/30/2012,9/16/2005,RESIGNED IN GOOD STANDING,OFFICER,MICHAEL RANDALL
11426,9996-9358,LARRY,ZIMMERMAN,11/19/1993,HUNTINGTON POLICE DEPARTMENT,SWORN/CERTIFIED,9/9/2022,8/19/1993,RETIRED IN GOOD STANDING,CAPTAIN,LARRY ZIMMERMAN
